<h1><strong>Configuração Inicial</strong></h1>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sb # statistical data visualization
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Changing configuration to display all columns of the data frames.
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 100)

<h1><strong>Analisando o dataset</strong></h1>

In [ ]:
df_calendar = pd.read_csv("/kaggle/input/eda-atenas-airbnb/calendar.csv")
df_listings = pd.read_csv("/kaggle/input/eda-atenas-airbnb/listings.csv")
df_reviews = pd.read_csv("/kaggle/input/eda-atenas-airbnb/reviews.csv")

In [ ]:
print("calendar shape: ", df_calendar.shape)
print("listings shape: ", df_listings.shape)
print("reviews shape: ", df_reviews.shape)

* calendar tem 3518603 linhas and 7 colunas
* listings tem 9640 linhas and 74 colunas
* reviews tem 366678 linhas and 6 colunas

In [ ]:
df_calendar.head(5)

In [ ]:
df_calendar.info()

A tabela calendar contem informação sobre a disponibilidade de cada propriedade durante um certo período de tempo. Ela também contém preço em moeda local e mínimo/máximo de noites que podem ser alugadas.

In [ ]:
df_reviews.head(5)

In [ ]:
df_reviews.info()

A tabela reviews contém dados sobre avaliações que os clientes dão a cada propriedade.

In [ ]:
df_listings.head(5)

A tabela listings contém dados detalhados a respeito das propriedades em si.

Algumas colunas possuem informações irrelevantes para esta análise, de modo que serão removidas.

As colunas "bathrooms", "calendar_updated" e "neighbourhood_group_cleansed" parecem estar vazias (somente valores NaN). Isso será verificado abaixo. Se for verdade, essas colunas também serão removidas.

In [ ]:
# Printa informações sobre o dataframe
df_listings.info()

In [ ]:
# Removes colunas inúteis
df_listings = df_listings.drop(columns=['listing_url', 'scrape_id', 'last_scraped', 'picture_url','host_url', 'host_picture_url', 'host_thumbnail_url','listing_url', 'neighbourhood_group_cleansed', 'bathrooms', 'calendar_updated'])

Os dados da coluna "price" são string (datatype do pandas é object), então isso será corrigido.

In [ ]:
# Remove o cifrão ao início da string
df_listings.price = df_listings.price.str.replace('$','')
# Remove a vírgula como separador de milhar e retorna o tipo como um float
df_listings.price = df_listings.price.str.replace(',','',regex='false').astype(float)

df_listings.price.dtypes

In [ ]:
df_listings.describe()

<h2>Investigação da distribuição de preços</h2>

* O desvio padrão dos preços está bastante alto (387 euros, para uma média de 90 euros)
* O percentil 75 dos preços é 80 euros, enquanto o máximo preço é 8 mil euros.

<h3>Investigando se a distribuição geográfica influencia no preço</h3>

In [ ]:
# Pega o percentil 99
price_99_percentile = df_listings.price.quantile(.99)
print(price_99_percentile)

# Plota um histograma do 1% mais caro
df_listings[df_listings.price >= price_99_percentile].price.hist()

In [ ]:
# Plota a posição geográfica das acomodações

# Mapa de Atenas exportado por openstreetmap.org
athens_m = plt.imread('../input/eda-atenas-airbnb/athens_map_corrected.png')

# Define a área de plotagem com base em coordenadas de latitude e longitude
BBox=(23.68,23.79,37.96,38.04)

plt.subplots(figsize=(20,15))
plt.scatter(df_listings[df_listings.price>=5000].longitude,df_listings[df_listings.price>=5000].latitude, s=30, c='red')
plt.scatter(df_listings[(df_listings.price<5000)&(df_listings.price>=3000)].longitude,df_listings[(df_listings.price<5000)&(df_listings.price>=3000)].latitude, s=30, c='blue')
plt.scatter(df_listings[(df_listings.price<3000)&(df_listings.price>=450)].longitude,df_listings[(df_listings.price<3000)&(df_listings.price>=450)].latitude, s=30, c='green')
plt.scatter(df_listings[df_listings.price<450].longitude,df_listings[df_listings.price<450].latitude, s=10, c='gray', alpha=0.35)
plt.title("Acomodações AirBNB em Atenas. Vermelho = acima de 5k euros/noite, Azul = entre 3k e 5k, Verde = entre 450 e 3k, Cinza = restante")
plt.legend()
plt.imshow(athens_m,zorder=0,extent=BBox)

* Há uma concentração maior de acomodações mais caras perto do centro da cidade. Isso faz sentido, pois essa é uma região turística, próxima da Acrópole e de outras regiões antigas, como o Templo de Zeus.
* No entanto, pode-se notar que o número de acomodações mais baratas também é grande nessas mesmas regiões. O bairro de Rouf (Ρουφ, a oeste da Acrópole) parece ser a única exceção, com menos pontos cinzas em relação aos pontos verdes.
* Estes fatos sugerem que o aumento de preço das acomodações 1% mais caras <strong>não é influenciado apenas pela localização.</strong>

In [ ]:
# Plota a posição geográfica das acomodações desconsiderando o 1% mais caro.

# Mapa de Atenas exportado por openstreetmap.org
athens_m = plt.imread('../input/eda-atenas-airbnb/athens_map_corrected.png')

# Define a área de plotagem com base em coordenadas de latitude e longitude
BBox=(23.68,23.79,37.96,38.04)

plt.subplots(figsize=(20,15))
plt.scatter(df_listings[(df_listings.price<450)&(df_listings.price>=185)].longitude,df_listings[(df_listings.price<450)&(df_listings.price>=185)].latitude, s=8, c='red', alpha=0.5)
plt.scatter(df_listings[(df_listings.price<185)&(df_listings.price>=80)].longitude,df_listings[(df_listings.price<185)&(df_listings.price>=80)].latitude, s=8, c='blue', alpha=0.5)
plt.scatter(df_listings[(df_listings.price<80)&(df_listings.price>=50)].longitude,df_listings[(df_listings.price<80)&(df_listings.price>=50)].latitude, s=8, c='green', alpha=0.5)
plt.scatter(df_listings[df_listings.price<50].longitude,df_listings[df_listings.price<50].latitude, s=8, c='gray', alpha=0.5)
plt.title("Acomodações AirBNB em Atenas. Vermelho = acima entre 185 e 450 euros/noite, Azul = entre 80 e 185, Verde = entre 50 e 80, Cinza = restante")
plt.legend()
plt.imshow(athens_m,zorder=0,extent=BBox)

In [ ]:
area = plt.gcf()
area.set_size_inches( 10, 15)


sb.boxplot(y = 'neighbourhood_cleansed', x='price', data=df_listings[df_listings.price<450], orient='h', 
            showfliers = False, width=1);

Desconsiderando as acomodações 1% mais caras e subdividindo as mais baratas, pode-se ver que a maior parte dos pontos azuis (entre 80 e 185 euros/noite) e vermelhos (entre 185 e 450 euros/noite) estão próximos à Acrópole. Mais longe desta região, as acomodações mais baratas (pontos cinzas) passam a ser majoritárias.
O bairro de Kolonaki (ΚΟΛΩΝΑΚΙ, nordeste da região da Acrópole) parece ser a região não adjacente à Acrópole que possui a maior quantidade de acomodações mais caras (mais pontos azuis e vermelhos em relação a pontos verdes e cinzas). Este é um bairro mais conhecido por suas lojas de grife e comércio mais luxuoso, o que pode justificar essa predominância maior.

<h3>Investigando a relação de outras variáveis com o preço</h3>

In [ ]:
# Há uma acomodação com 30 quartos e outras 2 com mais de 40 camas, o que dificulta a visualização, de modo que foram retiradas. Também foram mantidas apenas acomodações abaixo do percentil 99

df_listings_filtered = df_listings[(df_listings.price<450)&(df_listings.bedrooms<30)&(df_listings.beds<40)]

fig,a = plt.subplots(2,3,figsize=(20,15))
a[0,0].scatter(df_listings_filtered.bedrooms,df_listings_filtered.price,s=4)
a[0,0].set_title("Preço x Número de quartos")
a[0,1].scatter(df_listings_filtered.beds,df_listings_filtered.price,s=4)
a[0,1].set_title("Preço x Número de camas")
a[0,2].scatter(df_listings_filtered.accommodates,df_listings_filtered.price,s=4)
a[0,2].set_title("Preço x Número de ocupantes")
a[1,0].scatter(df_listings_filtered.review_scores_rating,df_listings_filtered.price,s=4)
a[1,0].set_title("Preço x Review Scores Rating")
a[1,1].scatter(df_listings_filtered.review_scores_location,df_listings_filtered.price,s=4)
a[1,1].set_title("Preço x Score de Avaliação da Localização")
a[1,2].scatter(df_listings_filtered.review_scores_value,df_listings_filtered.price,s=4)
a[1,2].set_title("Preço x Score de Avaliação do Valor")

As acomodações maiores (que possuem mais quartos, camas e comportam mais ocupantes) tendem a apresentar menos opções baratas do que as acomodações menores.

A maior parte das avaliações gerais (Review Scores Rating), avaliações de localização e avaliações de valor encontraram-se na faixa entre 4.0 e 5.0.
As acomodações mais caras apresentam avaliações mais altas nos três critérios, o que é evidenciado pela escassez de pontos na região superior dos gráficos para as notas entre 1.0 e 3.5, a menos do gráfico de avaliação de localização, que apresenta alguns valores entre 3.0 e 4.0 para acomodações acima de 200 euros/noite.
Houve apenas 1 nota menor do que 3.0 para acomodações acima de 200 euros/noite.

<h2>Analisando os textos dos comentários</h2>

In [ ]:
!pip install wordcloud

In [ ]:
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

In [ ]:
# Faz um inner join dos dataframes listings e reviews

df_listings_reviews = pd.merge(
    df_listings,
    df_reviews,
    how="inner",
    on=None,
    left_on="id",
    right_on="listing_id",
)

In [ ]:
df_listings.review_scores_rating.describe()

Primeiro, serão analisados os comentários das acomodações com os maiores ratings. Será usado o valor de 4.88 (mediana).

In [ ]:
# Concatena todos os textos das reviews de acomodações com rating > 4.88 (mediana), para uso com o wordcloud
high_review_text = " ".join(str(review) for review in df_listings_reviews.comments[df_listings_reviews.review_scores_rating>4.88])

print("Número de comentários em acomodações com rating maior que 4.88: ", len(df_listings_reviews.comments[df_listings_reviews.review_scores_rating>4.88]))

In [ ]:
# Cria uma imagem com wordcloud usando o texto concatenado acima

stopwords = set(STOPWORDS)

# Retirando algumas palavras que se repetem muito e dominam a nuvem, deixando a visualização das outras palavras prejudicada
stopwords.update(['apartment','και', 'Athen', 'Athens', 'br'])

wordcloud = WordCloud(stopwords=stopwords, background_color='white',width=800,height=400).generate(high_review_text)

plt.figure(figsize=(15,10))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

Podem-se ver diversas palavras positivas: "perfect", "clean", "comfortable", "highly recommend", "enjoyed", "amazing", "great", "friendly", entre outras.

Algumas palavras neutras indicam o que pode ser considerada uma boa característica: "close" (próximo a), "Acropoli", "metro", "metro station", "airport", "walking distance" etc.

As palavras mais frequentes foram "stay" e "place", possivelmente por serem comumente utilizadas em inglês em expressões como "nice stay" ou "great place".

Em seguida, são avaliados os comentários de acomodações com ratings menores. Utilizado o valor de 3.50 como referência.

In [ ]:
# Concatena todos os textos das reviews de acomodações com rating <= 3.50, para uso com o wordcloud
low_review_text = " ".join(str(review) for review in df_listings_reviews.comments[df_listings_reviews.review_scores_rating<=3.50])

print("Número de comentários em acomodações com rating menor que 3.50: ", len(df_listings_reviews.comments[df_listings_reviews.review_scores_rating<=3.50]))

In [ ]:
# Cria uma imagem com wordcloud usando o texto concatenado acima

stopwords = set(STOPWORDS)

# Retirando algumas palavras que se repetem muito e dominam a nuvem, deixando a visualização das outras palavras prejudicada
stopwords.update(['apartment','και', 'Athen', 'Athens', 'br', 'να', 'το'])

wordcloud = WordCloud(stopwords=stopwords, background_color='white',width=800,height=400).generate(low_review_text)

plt.figure(figsize=(15,10))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

Notam-se palavras de cunho positivo: "clean", "recommend", "great", "well", "good", "nice".

Por outro lado, notam-se também palavras de cunho negativo: "host canceled", "worst", "bad", "dirty", "problem".

Algumas palavras neutras podem dar ideia do que é considerado um ponto negativo: "arrival", "automated posting", "reservation days", "host", "experience", "hot water", "refund", "wifi", "money".

Podem-se ver mais palavras em grego neste caso. Algumas delas: "πολυ" (muito), "σπιτι" (lar), "καμια" (tudo), "δεν" (não), "τοποθεσια" (localização), "διαμερισμα" (apartamento), "καθαρο" (limpo)